# 获取课程表
## 导入相关库

In [1]:
import requests 
import json
from bs4 import BeautifulSoup

## 预先操作
1. 加密密码

In [2]:
def get_enc_password(origin_pwd: str) -> str:
    ENC_URL = 'http://cas.ecjtu.edu.cn/cas/loginPasswdEnc'
    enc_response = requests.post(ENC_URL, data={'pwd': origin_pwd})

    _ = enc_response.content.decode('utf8').replace("'", '"')
    return json.loads(_)['passwordEnc']

password = "your_password"
print(get_enc_password(password))

30052603102467EDE4DA3F7C61F023727039121D4EC611A63A537C2277D34897F693CC5142911946DFDD34206A55049E


2. 收集相关的api

In [3]:
ecjtu_login = "http://cas.ecjtu.edu.cn/cas/login" # 智慧交大登录页
jwxt_login = "https://jwxt.ecjtu.edu.cn/stuMag/Login_dcpLogin.action" # 教务系统登录页
ecjtu_to_jwxt = "http://cas.ecjtu.edu.cn/cas/login?service=https%3A%2F%2Fjwxt.ecjtu.edu.cn%2FstuMag%2FLogin_dcpLogin.action" #智慧交大登录教务系统
get_classes = "https://jwxt.ecjtu.edu.cn/Schedule/Weekcalendar_getTodayWeekcalendar.action" # 获取课程信息(post)
get_grade = "https://jwxt.ecjtu.edu.cn/scoreQuery/stuScoreQue_getStuScore.action?item=0401" # 获取成绩信息(get)

## 登录

In [4]:
def login(username:str,password:str,session:requests.Session) -> requests.Session:
    # 关闭警告
    requests.packages.urllib3.disable_warnings()
    # 获取密码
    enc = get_enc_password(password)
    # 打包登录body
    login_data = {
        'username': username,
        'password': enc,
        'lt': '',
        "service":"http://portal.ecjtu.edu.cn/dcp/index.jsp",
    }
    # 获取lt
    headers = {
        # 设置UA
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        # 设置Host
        "Host": "cas.ecjtu.edu.cn"
    }
    response = session.get(ecjtu_login,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    login_data['lt'] = soup.find('input', {'name': 'lt'})['value']
    # 登录
    headers_append = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Referer": "http://cas.ecjtu.edu.cn/cas/login"
      }
    headers.update(headers_append)
    response = session.post(ecjtu_login, data=login_data,headers=headers,allow_redirects=False)
    if not response.cookies.get('CASTGC'):
        print("账户或密码错误")
        return None
    # 获取教务JSESSIONID
    session.get(jwxt_login,headers=headers,allow_redirects=False,verify=False)
    # 获取教务跳转登录链接
    response_url = session.get(ecjtu_to_jwxt,allow_redirects=False)
    # 登录教务系统
    result = session.get(response_url.headers['Location'])
    if result.status_code != 200:
        print("教务系统错误，登录失败")
        return None
    print("登录成功")
    return session

session = requests.Session()
login("2022211003000112","20030115b",session)
classes = session.post(get_classes,data={"date":"2024-04-05"})
for k,v in classes.json().items():
    if k == "weekcalendarpojoList":
        for item in v:
            for i,j in item.items():
                print(i," ",j)
            print("\n")


登录成功
classString   1-2
courseRequire   必修课
teacherName   程俊峰
weekDay   5
classRoom   材料力学实验室(教9-202、113、114、结108)
classSpan   1,2
course   材料力学(B)
className   材料力学(B)(20232-1)
weekSpan   6
pkType   实验


classString   3-4
courseRequire   限选课
teacherName   王珏
weekDay   5
classRoom   31-311D
classSpan   3,4
course   Java程序设计(B)
className   Java程序设计(B)(20232-2)
weekSpan   1-16
pkType   上课


classString   5-6
courseRequire   必修课
teacherName   魏永丰
weekDay   5
classRoom   31-505
classSpan   5,6
course   数据库系统原理
className   数据库系统原理(20232-2)
weekSpan   1-16
pkType   上课


